# Prep data -- check and make it conform to the format of TMGs and downstream pipeline
- 


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import subprocess

In [ ]:
ddir = '/bigstore/Images2022/Gaby/dredFISH/DPNMF-R_8C_2022Oct18/fishdata'
# !ls $ddir/*.csv
f_mets = np.sort(glob.glob("*_metadata.csv", dir_fd=ddir))
f_mats = np.sort(glob.glob("*_matrix.csv",   dir_fd=ddir))
samples = np.array([f[:-len("_metadata.csv")] for f in f_mets])
assert len(f_mets) == len(f_mats)
assert len(f_mets) == len(samples)
f_mets, f_mats, samples

In [ ]:
outdir = "/bigstore/GeneralStorage/Data/dredFISH/DPNMF-R_8C_2022Oct18"
if not os.path.isdir(outdir):
    print(f"making {outdir}")
    os.mkdir(outdir)
    
for samp in samples:
    _dir = os.path.join(outdir, samp)
    if not os.path.isdir(_dir):
        print(f"making {_dir}")
        os.mkdir(_dir)
    

# house keeping
- copy samples over 
- rename metadata - stage_x, stage_y
- switch x and y
- add col section_index
- pixel_x, pixel_y, section_index
- remove hybe25 --> metadata

In [ ]:
for i, samp in enumerate(samples):
    print(samp)
    f = os.path.join(ddir, f"{samp}_matrix.csv")
    mat = pd.read_csv(f, index_col=0)
    
    f = os.path.join(ddir, f"{samp}_metadata.csv")
    meta = pd.read_csv(f, index_col=0)
    assert np.all(mat.index.values==meta.index.values)
    
    
    # add section_index
    meta['section_index'] = i+1 
    # add tmp_x, tmp_y -> ultimately to (stage_x, stage_y)
    meta[['tmp_x', 'tmp_y']] = meta[['pixel_y', 'pixel_x']]*1
    meta[['stage_x', 'stage_y']] = meta[['tmp_x', 'tmp_y']]
    # move polyT to metadata
    meta['polyt'] = mat['hybe25']
    mat = mat.drop('hybe25', axis=1)
    
    # copy over 
    fout_mat = os.path.join(outdir, samp, f"{samp}_matrix.csv")
    print(fout_mat)
    mat.to_csv(fout_mat)
    
    fout_meta = os.path.join(outdir, samp, f"{samp}_metadata.csv")
    print(fout_meta)
    meta.to_csv(fout_meta)
    
    # change mod
    subprocess.run(['chmod', '444', fout_mat])
    subprocess.run(['chmod', '444', fout_meta])

# checking x and y (and rotate it)

In [ ]:
from dredFISH.Visualization import viz_cell_layer
import importlib
import dredFISH
importlib.reload(dredFISH)
importlib.reload(dredFISH.Visualization)
importlib.reload(dredFISH.Visualization.viz_cell_layer)

In [ ]:
# mode = 'preview' # need a notebook to see; save nothing
# mode = 'view' # go from the beginning to the end
# mode = 'analysis-only' # analysis only; no plots

# mode = 'preview' # plot XY only; bypassing TMGs
mode = 'preview-save' # plot XY only, save results in prep for TMGs

sample_split_lines = {
    samples[0]: [[(10500, 2500), (13500,16000)]],
    samples[1]: [[(10300, 2500), (13500,16000)]],
    samples[2]: [[(10500, 2000), (12200,16000)]],
    samples[3]: [[(11000, 2500), (11000,16000)]],
}

# TMG
for samp in samples: #[[3]]:
    basepth = os.path.join(outdir, samp) # outdir samp
    
    # define a line to split things into hemi-coronal sections
    split_lines = sample_split_lines[samp]

    viz_cell_layer.main(mode, basepth, split_lines,)
    # break